In [1]:
from __future__ import unicode_literals
from sklearn.externals import joblib
from textacy import fileio, preprocess, Doc, vsm
import numpy as np

In [2]:
msg_model = joblib.load('/home/pretrain-model/category.pkl')
id2term_itr = fileio.read_json('/home/pretrain-model/category-id2term')
label2id_itr = fileio.read_json('/home/pretrain-model/category-label2id')

id2term = [i for i in id2term_itr][0]
label2id = [i for i in label2id_itr][0]

In [3]:
sample_msg = u'Referring to the letters, "Circle Line train incident: SMRT explains cause and steps taken" and "Commuters distressed and kept in the dark", forum writer Larry Chong Tuck Lai asked several questions regarding the safety of passengers when there is smoke inside the train. These are the questions, assuming a fully packed peak-hour train: How much smoke is needed before the smoke detector is activated and the ventilation system shut? What happens after the ventilation system is shut? How long will the air supply last in a crowded train with no air-conditioning or ventilation and with the presence of smoke in the cabin? While the incident was resolved within five minutes, what happens if we have a case where staff are unable to reach the site of the breakdown for hours, and the system is not restored? What options are available to passengers trapped in the train? Are there manual systems to allow passengers to free themselves, or at least open windows to allow air to flow in?CIC/CMC is working with SMRT Trains to prepare a response.'
#sample_msg = u'Taxi operators are taking the fight to Uber and Grab by setting up their own private-hire and limousine arms. The aim is to retain drivers who have begun to defect to the fast-expanding private-hire fleets run by the third-party taxi app providers. SMRT Corp has ordered a fleet of more than 300 new Toyota Vellfire MPVs, Corolla Altis sedans and some Lexus cars for that purpose. Taxi industry players noted that unlike cab companies, the newcomers are not required to meet service standards, their fleet expansion will not be restricted, they do not need to buy insurance policies that cover passengers, there is no restriction on vehicle age, and their cars attract far lower road taxes than cabs. Others said the fast-expanding third-party fleet will eventually impact cabbies\' earnings as they are competing for a fixed market that is not growing nearly as fast.'

In [4]:
sample_doc = Doc(sample_msg)

In [5]:
terms_lists_test = sample_doc.to_terms_list(ngrams={2, 3}, named_entities=True, as_strings=True)

In [6]:
dtm_test, i2t_test = vsm.doc_term_matrix([terms_lists_test], weighting='tfidf', normalize=True, smooth_idf=True, max_n_terms=100000)

In [7]:
dtm_test

<1x80 sparse matrix of type '<type 'numpy.float64'>'
	with 80 stored elements in Compressed Sparse Row format>

In [8]:
#build new index
test2train = dict()
for (i, j) in i2t_test.items():
    if j in id2term.values():
        train_i = id2term.keys()[id2term.values().index(j)]
        test2train[i]=train_i

In [9]:
test_feature = np.zeros((dtm_test.shape[0], len(id2term)))
dtm_test_array = dtm_test.toarray()

for (i, j) in test2train.items():
    test_feature[:, j] = dtm_test_array[:, i]

In [10]:
#show the prediction result
zip(range(test_feature.shape[0]), map(lambda x: label2id.keys()[label2id.values().index(x)], msg_model.predict(test_feature)))

[(0, u'smrt corporate')]

In [11]:
sample_msg

u"Taxi operators are taking the fight to Uber and Grab by setting up their own private-hire and limousine arms. The aim is to retain drivers who have begun to defect to the fast-expanding private-hire fleets run by the third-party taxi app providers. SMRT Corp has ordered a fleet of more than 300 new Toyota Vellfire MPVs, Corolla Altis sedans and some Lexus cars for that purpose. Taxi industry players noted that unlike cab companies, the newcomers are not required to meet service standards, their fleet expansion will not be restricted, they do not need to buy insurance policies that cover passengers, there is no restriction on vehicle age, and their cars attract far lower road taxes than cabs. Others said the fast-expanding third-party fleet will eventually impact cabbies' earnings as they are competing for a fixed market that is not growing nearly as fast."